In [1]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict
import os

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [2]:
rr = RandomRules(
  TOTAL_NUMBERS = 50,
  CHOOSE = 6,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5, 6],
  MAX_B_VALUES = [45, 46, 47, 48, 49, 50]
)

In [3]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=41'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [4]:
lst = []
post_str_b1 = None
post_str_b2 = None
post_str_b3 = None
post_str_b4 = None
post_str_b5 = None
post_str_blast = None
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_b2)
  cols.append(post_str_b3)
  cols.append(post_str_b4)
  cols.append(post_str_b5)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_b2 = str_combo[1]
  post_str_b3 = str_combo[2]
  post_str_b4 = str_combo[3]
  post_str_b5 = str_combo[4]
  post_str_blast = str_combo[-1]

with open('./gn_tnk_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas', 'yapa', 'adicionales', 'sorteo_extra',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
    'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_b5', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 6),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))
dfRows.write_parquet('dfrows_tnk.parquet')

TOTAL_LINES=2476


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""14/05/2025""","""1194""","""38 07 12 13 04 09""","""42""","""46 28""","""Promoción Sí o Sí""","""040709121338""","[4, 7, … 38]",4,7,9,12,13,38,"""04""","""07""","""09""","""12""","""13""","""38""",null,null,null,null,null,null,false,0,"""03_02_03_01_25""",34
"""11/05/2025""","""1193""","""46 21 49 45 03 43""","""17""","""20 30 32 44""","""Promoción Sí o Sí""","""032143454649""","[3, 21, … 49]",3,21,43,45,46,49,"""03""","""21""","""43""","""45""","""46""","""49""","""04""","""07""","""09""","""12""","""13""","""38""",false,0,"""18_22_02_01_03""",-59
"""07/05/2025""","""1192""","""48 02 38 50 33 35""","""49""","""20 16""","""Promoción Sí o Sí""","""023335384850""","[2, 33, … 50]",2,33,35,38,48,50,"""02""","""33""","""35""","""38""","""48""","""50""","""03""","""21""","""43""","""45""","""46""","""49""",true,16,"""31_02_03_10_02""",3
"""04/05/2025""","""1191""","""09 35 19 15 37 10""","""46""","""25 02 38""","""Promoción Sí o Sí""","""091015193537""","[9, 10, … 37]",9,10,15,19,35,37,"""09""","""10""","""15""","""19""","""35""","""37""","""02""","""33""","""35""","""38""","""48""","""50""",true,16,"""01_05_04_16_02""",-57
"""30/04/2025""","""1190""","""11 36 33 06 02 16""","""37""","""14 07 10 01""","""Promoción Sí o Sí""","""020611163336""","[2, 6, … 36]",2,6,11,16,33,36,"""02""","""06""","""11""","""16""","""33""","""36""","""09""","""10""","""15""","""19""","""35""","""37""",true,8,"""04_05_05_17_03""",-51
"""27/04/2025""","""1189""","""46 33 21 08 07 41""","""39""","""36 09 37""","""Promoción Sí o Sí""","""070821334146""","[7, 8, … 46]",7,8,21,33,41,46,"""07""","""08""","""21""","""33""","""41""","""46""","""02""","""06""","""11""","""16""","""33""","""36""",false,0,"""01_13_12_08_05""",-6
"""23/04/2025""","""1188""","""22 16 20 33 35 09""","""46""","""24""","""Promoción Sí o Sí""","""091620223335""","[9, 16, … 35]",9,16,20,22,33,35,"""09""","""16""","""20""","""22""","""33""","""35""","""07""","""08""","""21""","""33""","""41""","""46""",false,0,"""07_04_02_11_02""",-39
"""20/04/2025""","""1187""","""33 02 06 42 34 11""","""37""","""49 20""","""Promoción Sí o Sí""","""020611333442""","[2, 6, … 42]",2,6,11,33,34,42,"""02""","""06""","""11""","""33""","""34""","""42""","""09""","""16""","""20""","""22""","""33""","""35""",false,0,"""04_05_22_01_08""",120
"""16/04/2025""","""1186""","""14 31 46 38 48 12""","""15""","""39 30""","""Promoción Sí o Sí""","""121431384648""","[12, 14, … 48]",12,14,31,38,46,48,"""12""","""14""","""31""","""38""","""46""","""48""","""02""","""06""","""11""","""33""","""34""","""42""",false,0,"""02_17_07_08_02""",-54


In [5]:
dfRows.item(0, 'b2')

7

In [6]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str | None = None):
  if test_value == None:
    test_value = dfRows.item(0, col)
    print(f'Setting {test_value=}')

  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )

  mean_percent = df_viz['percent'].mean()
  mean_percent_75 = mean_percent * 75 / 50
  mean_percent_25 = mean_percent * 25 / 50

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart = chart * hv.HLine(mean_percent_75).opts(color='blue')
  chart = chart * hv.HLine(mean_percent_25).opts(color='blue')
  
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  df_unijump = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'unijump'
    ).agg(pl.count('unijump').alias('count')) \
    .sort("count", descending=True)


  print(f'df_viz FOR {col} = {test_value}')
  display(df_viz)

  display(chart)
  

  print(f'df_10_last FOR {col} = {test_value}')
  last_results = list(set(df_10_last[post_col][0:3]))
  print(f'Last 3 results {last_results=}')
  display(df_10_last)
  
  print(f'df_jumps_map FOR {col} = {test_value}')
  display(df_jumps_map)
  # display(df_unijump)

  print(f'df_unijump FOR {col} = {test_value}')
  display(df_unijump)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list() # Used to discard
  
  return lst_post_str_b, lst_jumps_map, last_results

In [7]:
possible_b1, exclude_jump_maps_b1, last_post_b1 = plot_post_col(col='str_b1', post_col='post_str_b1')

Setting test_value='04'
df_viz FOR str_b1 = 04


post_str_b1,count,percent
str,u32,f64
"""02""",30,13.824885
"""05""",26,11.981567
"""03""",22,10.138249
"""01""",22,10.138249
"""04""",20,9.21659
…,…,…
"""19""",2,0.921659
"""28""",1,0.460829
"""22""",1,0.460829


:Overlay
   .Bars.I    :Bars   [post_str_b1]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b1 = 04
Last 3 results last_results=['12', '03', '19']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""19/03/2025""","""1178""","""32 41 42 37 34 04""","""48""","""07 46 19""","""Promoción Sí o Sí""","""043234374142""","[4, 32, … 42]",4,32,34,37,41,42,"""04""","""32""","""34""","""37""","""41""","""42""","""03""","""07""","""18""","""24""","""37""","""49""",false,0,"""28_02_03_04_01""",23
"""02/02/2025""","""1165""","""20 48 14 12 28 04""","""02""","""40""","""Promoción Sí o Sí""","""041214202848""","[4, 12, … 48]",4,12,14,20,28,48,"""04""","""12""","""14""","""20""","""28""","""48""","""12""","""19""","""24""","""27""","""30""","""33""",true,20,"""08_02_06_08_20""",24
"""15/12/2024""","""1151""","""48 26 45 08 06 04""","""44""","""15 29""","""Promoción Sí o Sí""","""040608264548""","[4, 6, … 48]",4,6,8,26,45,48,"""04""","""06""","""08""","""26""","""45""","""48""","""19""","""22""","""28""","""33""","""35""","""47""",true,24,"""02_02_18_19_03""",29
"""11/12/2024""","""1150""","""43 04 39 31 10 46""","""06""","""36 07 22""","""Promoción Sí o Sí""","""041031394346""","[4, 10, … 46]",4,10,31,39,43,46,"""04""","""10""","""31""","""39""","""43""","""46""","""04""","""06""","""08""","""26""","""45""","""48""",true,16,"""06_21_08_04_03""",-43
"""22/09/2024""","""1127""","""20 37 48 17 38 04""","""45""","""01 44""","""Promoción Sí o Sí""","""041720373848""","[4, 17, … 48]",4,17,20,37,38,48,"""04""","""17""","""20""","""37""","""38""","""48""","""06""","""18""","""24""","""38""","""39""","""50""",false,0,"""13_03_17_01_10""",109
"""11/09/2024""","""1124""","""04 41 07 28 29 46""","""20""","""11""","""Promoción Sí o Sí""","""040728294146""","[4, 7, … 46]",4,7,28,29,41,46,"""04""","""07""","""28""","""29""","""41""","""46""","""01""","""03""","""29""","""36""","""38""","""43""",true,16,"""03_21_01_12_05""",-118
"""04/09/2024""","""1122""","""50 30 04 09 15 40""","""31""","""29 16 32""","""Promoción Sí o Sí""","""040915304050""","[4, 9, … 50]",4,9,15,30,40,50,"""04""","""09""","""15""","""30""","""40""","""50""","""07""","""14""","""15""","""18""","""23""","""35""",true,24,"""05_06_15_10_10""",41
"""01/09/2024""","""1121""","""23 50 41 19 04 37""","""28""","""09 06""","""Promoción Sí o Sí""","""041923374150""","[4, 19, … 50]",4,19,23,37,41,50,"""04""","""19""","""23""","""37""","""41""","""50""","""04""","""09""","""15""","""30""","""40""","""50""",true,1,"""15_04_14_04_09""",76
"""29/05/2024""","""1094""","""04 34 16 10 20 29""","""21""","""28 24 30""","""Promoción Sí o Sí""","""041016202934""","[4, 10, … 34]",4,10,16,20,29,34,"""04""","""10""","""16""","""20""","""29""","""34""","""02""","""08""","""12""","""13""","""26""","""37""",true,8,"""06_06_04_09_05""",-25


df_jumps_map FOR str_b1 = 04


jumps_map,count
str,u32
"""03_05_15_02_03""",1
"""10_03_10_02_14""",1
"""03_13_04_01_10""",1
"""05_03_02_17_03""",1
"""15_07_01_04_04""",1
…,…
"""05_03_02_15_10""",1
"""08_02_04_01_15""",1
"""04_04_05_08_18""",1


df_unijump FOR str_b1 = 04


unijump,count
i64,u32
24,5
-24,4
53,4
-9,4
29,4
…,…
108,1
-48,1
23,1


In [8]:
possible_b2, exclude_jump_maps_b2, last_post_b2 = plot_post_col(col='str_b2', post_col='post_str_b2')

Setting test_value='07'
df_viz FOR str_b2 = 07


post_str_b2,count,percent
str,u32,f64
"""05""",11,9.482759
"""10""",10,8.62069
"""03""",8,6.896552
"""07""",7,6.034483
"""11""",6,5.172414
…,…,…
"""18""",3,2.586207
"""26""",3,2.586207
"""25""",3,2.586207


:Overlay
   .Bars.I    :Bars   [post_str_b2]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b2 = 07
Last 3 results last_results=['05', '03', '08']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""23/03/2025""","""1179""","""24 07 18 03 37 49""","""43""","""50 28""","""Promoción Sí o Sí""","""030718243749""","[3, 7, … 49]",3,7,18,24,37,49,"""03""","""07""","""18""","""24""","""37""","""49""","""01""","""05""","""13""","""35""","""37""","""43""",false,0,"""04_11_06_13_12""",-44
"""11/09/2024""","""1124""","""04 41 07 28 29 46""","""20""","""11""","""Promoción Sí o Sí""","""040728294146""","[4, 7, … 46]",4,7,28,29,41,46,"""04""","""07""","""28""","""29""","""41""","""46""","""01""","""03""","""29""","""36""","""38""","""43""",true,16,"""03_21_01_12_05""",-118
"""09/06/2024""","""1097""","""39 20 29 03 07 11""","""27""","""21 08""","""Promoción Sí o Sí""","""030711202939""","[3, 7, … 39]",3,7,11,20,29,39,"""03""","""07""","""11""","""20""","""29""","""39""","""03""","""08""","""11""","""16""","""32""","""48""",true,9,"""04_04_09_09_10""",16
"""14/04/2024""","""1081""","""05 17 07 47 41 19""","""04""","""33 03""","""Promoción Sí o Sí""","""050717194147""","[5, 7, … 47]",5,7,17,19,41,47,"""05""","""07""","""17""","""19""","""41""","""47""","""06""","""21""","""26""","""27""","""40""","""47""",true,19,"""02_10_02_22_06""",-108
"""03/04/2024""","""1078""","""02 07 38 18 31 26""","""17""","""41 03""","""Promoción Sí o Sí""","""020718263138""","[2, 7, … 38]",2,7,18,26,31,38,"""02""","""07""","""18""","""26""","""31""","""38""","""05""","""10""","""20""","""22""","""25""","""35""",false,0,"""05_11_08_05_07""",-4
"""06/03/2024""","""1070""","""30 22 04 07 39 37""","""38""","""43 02 18""","""Promoción Sí o Sí""","""040722303739""","[4, 7, … 39]",4,7,22,30,37,39,"""04""","""07""","""22""","""30""","""37""","""39""","""12""","""18""","""30""","""32""","""34""","""35""",true,16,"""03_15_08_07_02""",-35
"""14/02/2024""","""1064""","""42 03 18 07 29 39""","""30""","""28 31""","""Promoción Sí o Sí""","""030718293942""","[3, 7, … 42]",3,7,18,29,39,42,"""03""","""07""","""18""","""29""","""39""","""42""","""12""","""23""","""24""","""31""","""35""","""42""",true,8,"""04_11_11_10_03""",-11
"""17/09/2023""","""1021""","""36 05 47 46 07 08""","""14""","""02 01 18""","""Promoción Sí o Sí""","""050708364647""","[5, 7, … 47]",5,7,8,36,46,47,"""05""","""07""","""08""","""36""","""46""","""47""","""19""","""24""","""27""","""34""","""42""","""47""",true,16,"""02_01_28_10_01""",127
"""10/09/2023""","""1019""","""34 41 47 07 20 03""","""16""","""22""","""Promoción Sí o Sí""","""030720344147""","[3, 7, … 47]",3,7,20,34,41,47,"""03""","""07""","""20""","""34""","""41""","""47""","""19""","""24""","""28""","""41""","""42""","""48""",true,1,"""04_13_14_07_06""",14


df_jumps_map FOR str_b2 = 07


jumps_map,count
str,u32
"""01_04_05_23_05""",2
"""01_03_13_15_05""",1
"""06_05_06_02_23""",1
"""03_07_02_08_07""",1
"""01_07_09_04_01""",1
…,…
"""04_01_26_01_02""",1
"""04_01_26_03_05""",1
"""03_03_02_15_01""",1


df_unijump FOR str_b2 = 07


unijump,count
i64,u32
16,4
-64,3
12,2
-4,2
-8,2
…,…
4,1
-92,1
11,1


In [9]:
possible_b3, exclude_jump_maps_b3, last_post_b3 = plot_post_col(col='str_b3', post_col='post_str_b3')

Setting test_value='09'
df_viz FOR str_b3 = 09


post_str_b3,count,percent
str,u32,f64
"""25""",5,11.904762
"""14""",4,9.52381
"""16""",4,9.52381
"""13""",3,7.142857
"""19""",3,7.142857
…,…,…
"""15""",1,2.380952
"""33""",1,2.380952
"""10""",1,2.380952


:Overlay
   .Bars.I    :Bars   [post_str_b3]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b3 = 09
Last 3 results last_results=['13', '24', '34']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""26/06/2024""","""1102""","""43 06 14 01 09 11""","""30""","""17 42 45 20""","""Promoción Sí o Sí""","""010609111443""","[1, 6, … 43]",1,6,9,11,14,43,"""01""","""06""","""09""","""11""","""14""","""43""","""13""","""15""","""34""","""38""","""42""","""48""",true,16,"""05_03_02_03_29""",22
"""30/11/2022""","""0938""","""33 41 09 01 05 17""","""35""","""02 13 36""","""Promoción Sí o Sí""","""010509173341""","[1, 5, … 41]",1,5,9,17,33,41,"""01""","""05""","""09""","""17""","""33""","""41""","""05""","""14""","""24""","""27""","""33""","""40""",true,26,"""04_04_08_16_08""",-20
"""05/12/2021""","""0835""","""01 06 16 29 17 09""","""41""","""44 04""","""Promoción Sí o Sí""","""010609161729""","[1, 6, … 29]",1,6,9,16,17,29,"""01""","""06""","""09""","""16""","""17""","""29""","""01""","""10""","""13""","""16""","""19""","""42""",true,16,"""05_03_07_01_12""",43
"""31/03/2021""","""0764""","""09 10 05 01 16 39""","""36""","""""","""""","""010509101639""","[1, 5, … 39]",1,5,9,10,16,39,"""01""","""05""","""09""","""10""","""16""","""39""","""04""","""15""","""17""","""26""","""31""","""41""",true,24,"""04_04_01_06_23""",-7
"""01/03/2020""","""0684""","""08 06 27 09 33 26""","""17""","""""","""""","""060809262733""","[6, 8, … 33]",6,8,9,26,27,33,"""06""","""08""","""09""","""26""","""27""","""33""","""09""","""12""","""19""","""21""","""30""","""43""",true,16,"""02_01_17_01_06""",102
"""19/01/2020""","""0672""","""06 02 12 18 23 09""","""01""","""""","""""","""020609121823""","[2, 6, … 23]",2,6,9,12,18,23,"""02""","""06""","""09""","""12""","""18""","""23""","""05""","""11""","""34""","""35""","""40""","""42""",true,8,"""04_03_03_06_05""",-9
"""19/12/2018""","""0559""","""09 03 23 40 33 05""","""08""","""""","""""","""030509233340""","[3, 5, … 40]",3,5,9,23,33,40,"""03""","""05""","""09""","""23""","""33""","""40""","""02""","""12""","""16""","""21""","""35""","""45""",true,16,"""02_04_14_10_07""",37
"""08/10/2017""","""0434""","""26 05 24 35 09 06""","""32""","""""","""""","""050609242635""","[5, 6, … 35]",5,6,9,24,26,35,"""05""","""06""","""09""","""24""","""26""","""35""","""13""","""19""","""25""","""30""","""40""","""44""",true,16,"""01_03_15_02_09""",80
"""03/09/2017""","""0424""","""31 09 43 01 03 44""","""06""","""""","""""","""010309314344""","[1, 3, … 44]",1,3,9,31,43,44,"""01""","""03""","""09""","""31""","""43""","""44""","""09""","""13""","""17""","""21""","""36""","""37""",true,16,"""02_06_22_12_01""",63


df_jumps_map FOR str_b3 = 09


jumps_map,count
str,u32
"""06_01_02_11_16""",1
"""02_02_05_04_08""",1
"""05_01_05_01_07""",1
"""01_03_15_02_09""",1
"""01_07_14_07_13""",1
…,…
"""03_04_26_03_03""",1
"""01_02_01_06_23""",1
"""04_03_05_20_08""",1


df_unijump FOR str_b3 = 09


unijump,count
i64,u32
15,2
-13,2
-8,1
92,1
22,1
…,…
-50,1
-3,1
-39,1


In [10]:
possible_b4, exclude_jump_maps_b4, last_post_b4 = plot_post_col(col='str_b4', post_col='post_str_b4')

Setting test_value='12'
df_viz FOR str_b4 = 12


post_str_b4,count,percent
str,u32,f64
"""19""",4,11.764706
"""35""",4,11.764706
"""27""",3,8.823529
"""30""",3,8.823529
"""20""",3,8.823529
…,…,…
"""29""",1,2.941176
"""17""",1,2.941176
"""22""",1,2.941176


:Overlay
   .Bars.I    :Bars   [post_str_b4]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b4 = 12
Last 3 results last_results=['42', '27', '28']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""24/07/2022""","""0901""","""07 23 04 12 03 31""","""22""","""41""","""Promoción Sí o Sí""","""030407122331""","[3, 4, … 31]",3,4,7,12,23,31,"""03""","""04""","""07""","""12""","""23""","""31""","""02""","""06""","""11""","""27""","""29""","""31""",true,1,"""01_03_05_11_08""",-17
"""22/05/2022""","""0883""","""11 10 12 21 01 26""","""36""","""33 38""","""Promoción Sí o Sí""","""011011122126""","[1, 10, … 26]",1,10,11,12,21,26,"""01""","""10""","""11""","""12""","""21""","""26""","""12""","""28""","""39""","""42""","""43""","""45""",true,24,"""09_01_01_09_05""",-20
"""17/11/2021""","""0830""","""02 03 10 30 36 12""","""06""","""""","""""","""020310123036""","[2, 3, … 36]",2,3,10,12,30,36,"""02""","""03""","""10""","""12""","""30""","""36""","""15""","""24""","""26""","""28""","""38""","""44""",false,0,"""01_07_02_18_06""",-81
"""19/01/2020""","""0672""","""06 02 12 18 23 09""","""01""","""""","""""","""020609121823""","[2, 6, … 23]",2,6,9,12,18,23,"""02""","""06""","""09""","""12""","""18""","""23""","""05""","""11""","""34""","""35""","""40""","""42""",true,8,"""04_03_03_06_05""",-9
"""17/03/2019""","""0584""","""24 07 41 11 04 12""","""18""","""""","""""","""040711122441""","[4, 7, … 41]",4,7,11,12,24,41,"""04""","""07""","""11""","""12""","""24""","""41""","""12""","""13""","""15""","""16""","""43""","""44""",true,16,"""03_04_01_12_17""",-38
"""27/05/2018""","""0500""","""12 28 05 10 14 01""","""42""","""""","""""","""010510121428""","[1, 5, … 28]",1,5,10,12,14,28,"""01""","""05""","""10""","""12""","""14""","""28""","""09""","""13""","""14""","""20""","""22""","""26""",true,24,"""04_05_02_02_14""",2
"""12/06/2016""","""0296""","""01 08 11 12 15 36""","""22""","""""","""""","""010811121536""","[1, 8, … 36]",1,8,11,12,15,36,"""01""","""08""","""11""","""12""","""15""","""36""","""06""","""20""","""32""","""35""","""43""","""45""",false,0,"""07_03_01_03_21""",10
"""25/05/2016""","""0291""","""05 08 11 12 33 44""","""45""","""""","""""","""050811123344""","[5, 8, … 44]",5,8,11,12,33,44,"""05""","""08""","""11""","""12""","""33""","""44""","""02""","""17""","""19""","""27""","""37""","""42""",true,8,"""03_03_01_21_11""",-76
"""17/01/2016""","""0254""","""01 05 06 12 24 40""","""19""","""""","""""","""010506122440""","[1, 5, … 40]",1,5,6,12,24,40,"""01""","""05""","""06""","""12""","""24""","""40""","""12""","""18""","""21""","""30""","""40""","""45""",true,16,"""04_01_06_12_16""",4


df_jumps_map FOR str_b4 = 12


jumps_map,count
str,u32
"""03_03_02_15_01""",1
"""01_03_05_11_08""",1
"""01_03_04_09_13""",1
"""07_03_01_03_21""",1
"""04_03_03_06_05""",1
…,…
"""02_03_05_16_08""",1
"""02_03_01_11_05""",1
"""02_03_06_10_20""",1


df_unijump FOR str_b4 = 12


unijump,count
i64,u32
-36,2
-20,2
-43,2
31,1
20,1
…,…
-82,1
-76,1
28,1


In [11]:
possible_b5, exclude_jump_maps_b5, last_post_b5 = plot_post_col(col='str_b5', post_col='post_str_b5')

Setting test_value='13'
df_viz FOR str_b5 = 13


post_str_b5,count,percent
str,u32,f64
"""20""",1,33.333333
"""28""",1,33.333333
"""33""",1,33.333333


:Overlay
   .Bars.I    :Bars   [post_str_b5]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b5 = 13
Last 3 results last_results=['28', '20', '33']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""27/06/2010""","""1140""","""05 08 09 10 13 35""","""25""","""""","""""","""050809101335""","[5, 8, … 35]",5,8,9,10,13,35,"""05""","""08""","""09""","""10""","""13""","""35""","""05""","""06""","""25""","""31""","""33""","""37""",true,24,"""03_01_01_03_22""",15
"""13/04/1997""","""0131""","""04 07 08 12 13 22""","""""","""""","""""","""040708121322""","[4, 7, … 22]",4,7,8,12,13,22,"""04""","""07""","""08""","""12""","""13""","""22""","""03""","""04""","""10""","""19""","""20""","""28""",false,0,"""03_01_04_01_09""",28
"""18/08/1996""","""0097""","""01 04 10 12 13 27""","""""","""""","""""","""010410121327""","[1, 4, … 27]",1,4,10,12,13,27,"""01""","""04""","""10""","""12""","""13""","""27""","""01""","""09""","""14""","""19""","""28""","""30""",false,0,"""03_06_02_01_14""",1


df_jumps_map FOR str_b5 = 13


jumps_map,count
str,u32
"""03_06_02_01_14""",1
"""03_01_01_03_22""",1
"""03_01_04_01_09""",1


df_unijump FOR str_b5 = 13


unijump,count
i64,u32
1,1
15,1
28,1


In [12]:
possible_blast, exclude_jump_maps_blast, last_post_blast = plot_post_col(col='str_b6', post_col='post_str_blast')

Setting test_value='38'
df_viz FOR str_b6 = 38


post_str_blast,count,percent
str,u32,f64
"""44""",15,12.5
"""43""",11,9.166667
"""42""",11,9.166667
"""36""",9,7.5
"""45""",8,6.666667
…,…,…
"""30""",2,1.666667
"""31""",1,0.833333
"""25""",1,0.833333


:Overlay
   .Bars.I    :Bars   [post_str_blast]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b6 = 38
Last 3 results last_results=['26', '50', '45']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""04/12/2024""","""1148""","""17 29 20 38 02 36""","""26""","""33 01 05""","""Promoción Sí o Sí""","""021720293638""","[2, 17, … 38]",2,17,20,29,36,38,"""02""","""17""","""20""","""29""","""36""","""38""","""08""","""17""","""26""","""43""","""44""","""45""",false,0,"""15_03_09_07_02""",31
"""07/08/2024""","""1114""","""10 38 36 18 32 14""","""33""","""31 29 42 28""","""Promoción Sí o Sí""","""101418323638""","[10, 14, … 38]",10,14,18,32,36,38,"""10""","""14""","""18""","""32""","""36""","""38""","""12""","""18""","""38""","""43""","""47""","""50""",true,12,"""04_04_14_04_02""",58
"""24/07/2024""","""1110""","""23 32 28 19 38 25""","""40""","""03""","""Promoción Sí o Sí""","""192325283238""","[19, 23, … 38]",19,23,25,28,32,38,"""19""","""23""","""25""","""28""","""32""","""38""","""03""","""08""","""14""","""19""","""25""","""26""",false,0,"""04_02_03_04_06""",4
"""03/04/2024""","""1078""","""02 07 38 18 31 26""","""17""","""41 03""","""Promoción Sí o Sí""","""020718263138""","[2, 7, … 38]",2,7,18,26,31,38,"""02""","""07""","""18""","""26""","""31""","""38""","""05""","""10""","""20""","""22""","""25""","""35""",false,0,"""05_11_08_05_07""",-4
"""17/12/2023""","""1047""","""05 22 11 17 38 10""","""14""","""07 41 16""","""Promoción Sí o Sí""","""051011172238""","[5, 10, … 38]",5,10,11,17,22,38,"""05""","""10""","""11""","""17""","""22""","""38""","""07""","""09""","""11""","""21""","""27""","""44""",false,0,"""05_01_06_05_16""",33
"""25/10/2023""","""1032""","""38 17 22 26 21 36""","""07""","""27 29 11""","""Promoción Sí o Sí""","""172122263638""","[17, 21, … 38]",17,21,22,26,36,38,"""17""","""21""","""22""","""26""","""36""","""38""","""06""","""10""","""11""","""19""","""36""","""39""",false,0,"""04_01_04_10_02""",-14
"""27/08/2023""","""1015""","""20 22 38 28 16 14""","""36""","""34 43 08""","""Promoción Sí o Sí""","""141620222838""","[14, 16, … 38]",14,16,20,22,28,38,"""14""","""16""","""20""","""22""","""28""","""38""","""01""","""12""","""25""","""28""","""33""","""44""",true,4,"""02_04_02_06_10""",-16
"""19/07/2023""","""1004""","""06 38 35 23 09 31""","""10""","""16 04 47""","""Promoción Sí o Sí""","""060923313538""","[6, 9, … 38]",6,9,23,31,35,38,"""06""","""09""","""23""","""31""","""35""","""38""","""09""","""15""","""26""","""31""","""38""","""43""",false,0,"""03_14_08_04_03""",-18
"""09/07/2023""","""1001""","""26 38 29 21 06 11""","""45""","""15 44 05""","""Promoción Sí o Sí""","""061121262938""","[6, 11, … 38]",6,11,21,26,29,38,"""06""","""11""","""21""","""26""","""29""","""38""","""01""","""13""","""20""","""23""","""27""","""42""",false,0,"""05_10_05_03_09""",-8


df_jumps_map FOR str_b6 = 38


jumps_map,count
str,u32
"""07_09_02_07_10""",1
"""03_08_05_09_05""",1
"""05_02_01_17_01""",1
"""02_04_02_06_10""",1
"""10_03_08_11_02""",1
…,…
"""04_01_01_07_07""",1
"""19_03_02_05_04""",1
"""07_07_03_03_12""",1


df_unijump FOR str_b6 = 38


unijump,count
i64,u32
46,4
-16,3
27,3
4,3
-30,2
…,…
3,1
58,1
29,1


In [13]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [14]:
parquet_file = './parquet_tnk_v3.parquet'

# check if parquet_file exists
if not os.path.exists(parquet_file):
  print(f'{parquet_file=} does not exists. creating')
  lst = []
  post_str_b1 = None
  post_str_blast = None
  for row in combos:
    cols = []
    str_row = " ".join([str(r).zfill(2) for r in row])
    id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    cols.extend(str_combo)
    cols.append(post_str_b1)
    cols.append(post_str_blast)

    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)

    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)

    post_str_b1 = str_combo[0]
    post_str_blast = str_combo[-1]

  df_Orows = pl.DataFrame(
    lst,
    schema=[
      # 'fecha', 'sorteo', 'bolillas',
      'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
      'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
      'post_str_b1', 'post_str_blast',
      'is_rare_combination', 'reason',
      "jumps_map", "unijump"
    ],
    schema_overrides={
      'combo': pl.Array(pl.UInt8, 6),
      'b1': pl.UInt8,
      'b2': pl.UInt8,
      'b3': pl.UInt8,
      'b4': pl.UInt8,
      'b5': pl.UInt8,
      'b6': pl.UInt8,
      'reason': pl.UInt8
    },
    orient='row'
  )

  df_Orows.write_parquet(parquet_file)
  TOTAL_LINES = len(df_Orows)
  print(f'{TOTAL_LINES=} loaded to {parquet_file=}')

  display(df_Orows.head(10))
else:
  df_Orows = pl.read_parquet(parquet_file)

In [15]:
# possible_b1, exclude_jump_maps_b1
df_possible = df_Orows.filter(
  pl.col("str_b1").is_in(possible_b1),
  pl.col("str_b2").is_in(possible_b2),
  pl.col("str_b3").is_in(possible_b3),
  pl.col("str_b4").is_in(possible_b4),
  pl.col("str_b5").is_in(possible_b5),
  pl.col("str_b6").is_in(possible_blast),
  
  ~pl.col("str_b1").is_in(last_post_b1),
  # ~pl.col("str_b2").is_in(last_post_b2),
  # ~pl.col("str_b3").is_in(last_post_b3),
  # ~pl.col("str_b4").is_in(last_post_b4),
  # ~pl.col("str_b5").is_in(last_post_b5),
  ~pl.col("str_b6").is_in(last_post_blast),

  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_b2 + exclude_jump_maps_b3 + exclude_jump_maps_b4 + exclude_jump_maps_b5 + exclude_jump_maps_blast),
  # ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_b2 + exclude_jump_maps_blast),
  pl.col("is_rare_combination") == False
)


display(df_possible)

id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""010313162034""","[1, 3, … 34]",1,3,13,16,20,34,"""01""","""03""","""13""","""16""","""20""","""34""","""01""","""33""",false,0,"""02_10_03_04_14""",-22
"""010313162036""","[1, 3, … 36]",1,3,13,16,20,36,"""01""","""03""","""13""","""16""","""20""","""36""","""01""","""35""",false,0,"""02_10_03_04_16""",-20
"""010313162039""","[1, 3, … 39]",1,3,13,16,20,39,"""01""","""03""","""13""","""16""","""20""","""39""","""01""","""38""",false,0,"""02_10_03_04_19""",-17
"""010313162040""","[1, 3, … 40]",1,3,13,16,20,40,"""01""","""03""","""13""","""16""","""20""","""40""","""01""","""39""",false,0,"""02_10_03_04_20""",-16
"""010313162041""","[1, 3, … 41]",1,3,13,16,20,41,"""01""","""03""","""13""","""16""","""20""","""41""","""01""","""40""",false,0,"""02_10_03_04_21""",-15
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""071525303342""","[7, 15, … 42]",7,15,25,30,33,42,"""07""","""15""","""25""","""30""","""33""","""42""","""07""","""41""",false,0,"""08_10_05_03_09""",-5
"""071525303343""","[7, 15, … 43]",7,15,25,30,33,43,"""07""","""15""","""25""","""30""","""33""","""43""","""07""","""42""",false,0,"""08_10_05_03_10""",-4
"""071525303344""","[7, 15, … 44]",7,15,25,30,33,44,"""07""","""15""","""25""","""30""","""33""","""44""","""07""","""43""",false,0,"""08_10_05_03_11""",-3


In [35]:
df_possible.sample(n=1)


id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""020517203336""","[2, 5, … 36]",2,5,17,20,33,36,"""02""","""05""","""17""","""20""","""33""","""36""","""02""","""35""",false,0,"""03_12_03_13_03""",-76
